# Simulation 7

**Mengqi Liu**

**Oct 9， 2023**

In [ ]:
import numpy as np
import utils
import linear
import pandas as pd
import scipy
import math
import scipy
import matplotlib.pyplot as plt
from itertools import permutations
import random
import seaborn as sns
import linear2

1. Correction for identification problem
2. What is the learned graph? - continuous and discrete
3. formalize for logistic scenarios
4. Use SGLD for solving DAGMA (add some randomness)
